In [7]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [8]:
!pip install langchain_community langchain_openai faiss-cpu

## Modelos

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [10]:
response = llm.invoke("Olá")

print(response)

content='Olá! Como posso ajudar você hoje?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 8, 'total_tokens': 16}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'stop', 'logprobs': None} id='run-06cf655a-9ad5-4cc7-a06e-3b2b72e934ff-0' usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16}


## Prompts

### Templates Simples

In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

prompt = prompt_template.invoke({"texto": "Artificial Intelligence is the future!"})
print(prompt)

messages=[HumanMessage(content='Traduza o seguinte texto para português: Artificial Intelligence is the future!')]


In [12]:
response = llm.invoke(prompt)

print(response.content)

A inteligência artificial é o futuro!


### Templates de Mensagens

In [13]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

# messages = [
#     ("system", "Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
#     ("human", "Hello, how are you?"),
# ]

response = llm.invoke(messages)

print(response)

content='Olá, como você está?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 36, 'total_tokens': 42}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f905cf32a9', 'finish_reason': 'stop', 'logprobs': None} id='run-6d01c262-a6f5-4332-bf4d-a182e7b1a5e7-0' usage_metadata={'input_tokens': 36, 'output_tokens': 6, 'total_tokens': 42}


In [14]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [15]:
prompt = prompt_template.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "português",
    "texto": "Hello, how are you?"
})

print(prompt)

messages=[SystemMessage(content='Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas.'), HumanMessage(content='Hello, how are you?')]


In [16]:
response = llm.invoke(prompt)

print(response.content)

Olá, como você está?


### Parsers

In [17]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [18]:
output = parser.invoke(response)

print(output)

Olá, como você está?


## Encadeamento

In [19]:
chain = prompt_template | llm | parser

In [20]:
response = chain.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife tem tubarões!"
})

print(response)

¡Las playas de Recife tienen tiburones!


In [21]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [22]:
output = translate("The beaches of Recife have sharks!", "inglês", "espanhol")

print(output)

¡Las playas de Recife tienen tiburones!


## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

In [65]:
# Importando as bibliotecas necessárias
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser



In [66]:
# Inicializando o modelo
llm = ChatOpenAI(model="gpt-4o-mini")

# Criando o template para gerar uma piada
prompt_template = ChatPromptTemplate.from_template("Crie uma piada sobre: {topico}")


In [67]:
# Criando a chain
chain = prompt_template | llm | StrOutputParser()

# Função para criar uma piada
def criar_piada(topico):
    response = chain.invoke({"topico": topico})
    return response


In [ ]:

# Testando a função
output = criar_piada("programadores")
print(output)

Por que os programadores preferem o escuro?

Porque no modo noturno, eles conseguem encontrar os bugs mais facilmente!


### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

In [33]:
# Criando o template para classificação de sentimento
prompt_template = ChatPromptTemplate.from_template("Classifique o sentimento do seguinte texto como positivo, neutro ou negativo: {texto}")


In [34]:
# Criando a chain
chain = prompt_template | llm | StrOutputParser()

# Função para classificar o sentimento
def classificar_sentimento(texto):
    response = chain.invoke({"texto": texto})
    return response


In [36]:
# Testando a função
output = classificar_sentimento("É razoavel programar em Python!")
print(output)


O sentimento do texto "É razoável programar em Python!" pode ser classificado como neutro. A palavra "razoável" sugere uma opinião moderada, sem um entusiasmo forte nem uma crítica negativa.


### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

In [37]:
# Criando o template para gerar código Python
prompt_template = ChatPromptTemplate.from_template("Gere o código de uma função Python que: {descricao}")


In [38]:
# Criando a chain
chain = prompt_template | llm | StrOutputParser()

# Função para gerar código Python
def gerar_codigo(descricao):
    response = chain.invoke({"descricao": descricao})
    return response


In [40]:
# Testando a função
output = gerar_codigo("calcule a soma de dois números")
print(output)

Aqui está um exemplo de uma função em Python que calcula a soma de dois números:

```python
def soma(a, b):
    return a + b

# Exemplo de uso da função
numero1 = 5
numero2 = 7
resultado = soma(numero1, numero2)
print(f"A soma de {numero1} e {numero2} é {resultado}.")
```

Essa função `soma` recebe dois parâmetros, `a` e `b`, e retorna a soma deles. No exemplo de uso, a função é chamada com os números 5 e 7, e o resultado é impresso.


### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

In [54]:
# Template para explicar o tópico
explain_template = ChatPromptTemplate.from_template("Explique de forma simplificada o seguinte tópico: {topico}")

# Template para traduzir para inglês
translate_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para mandarim: {texto}")

In [55]:
# Chain para explicar o tópico
explain_chain = explain_template | llm | StrOutputParser()

# Chain para traduzir para inglês
translate_chain = translate_template | llm | StrOutputParser()


In [56]:
# Função para explicar e traduzir o tópico
def explicar_e_traduzir(topico):
    explicacao = explain_chain.invoke({"topico": topico})
    traducao = translate_chain.invoke({"texto": explicacao})
    return traducao


In [57]:
# Testando a função
output = explicar_e_traduzir("Inteligência Artificial")
print(output)


人工智能（AI）是计算机科学的一个分支，旨在创建能够模拟人类智能的机器和系统。这意味着这些机器可以自主学习、推理、解决问题和做出决策，也就是说，无需直接的人类干预。

人工智能可以分为不同类型：

1. **弱人工智能**：设计用于执行特定任务，例如虚拟助手（如：Siri、Alexa）或推荐系统（如：Netflix、Spotify）。它们没有意识或理解能力，仅仅遵循既定的算法。

2. **强人工智能**：理论上，它将是一种能够理解、学习和应用知识，类似于人类的人工智能。这种类型的人工智能尚未实现。

人工智能利用机器学习等技术，在这些技术中，算法分析数据并随着时间的推移提高其性能，以及神经网络，这些网络受到人类大脑工作原理的启发。

在日常生活中，人工智能广泛应用于语音识别、医疗诊断、自动驾驶汽车等多个领域，带来了效率和自动化的好处。然而，它也引发了伦理问题以及对隐私和安全的担忧。


### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.

In [58]:
# Criando o template para responder perguntas sobre o CESAR School
prompt_template = ChatPromptTemplate.from_template("Responda à seguinte pergunta sobre o CESAR School: {pergunta}")




In [60]:
# Criando a chain
chain = prompt_template | llm | StrOutputParser()


In [61]:
# Função para responder perguntas sobre o CESAR School
def responder_pergunta(pergunta):
    response = chain.invoke({"pergunta": pergunta})
    return response


In [62]:
# Testando a função
output = responder_pergunta("Quais cursos são oferecidos pelo CESAR School?")
print(output)

O CESAR School oferece uma variedade de cursos focados em tecnologia, design e inovação. Entre os principais programas, estão:

1. **Graduação**: Cursos de bacharelado em áreas como Engenharia de Software, Design de Interação e Análise e Desenvolvimento de Sistemas.

2. **Pós-graduação**: Especializações em temas como Gestão de Projetos, Design, Ciência de Dados, Inteligência Artificial, entre outros.

3. **Cursos livres e de curta duração**: Oferecem capacitação em tópicos específicos, como programação, UX/UI, e metodologias ágeis.

4. **Bootcamps**: Programas intensivos e imersivos focados em áreas como desenvolvimento de software e data science.

É sempre bom conferir diretamente no site do CESAR School para informações atualizadas sobre a oferta de cursos, pois eles podem variar ao longo do tempo.
